In [1]:
import numpy as np
from scipy.stats import norm, gamma
import matplotlib.pyplot as plt
#from gamma_copula_dynamic_y_markov_window import fit_y_markov_model, conditional_density_given_Y, sample_given_Y
#from gamma_copula_dynamic_simple import fit_gamma_marginals, transform_to_z, estimate_covariance


In [2]:
def simulate_iid_gamma_gaussian(K,N, d, alpha, beta, R):
    """K : number of samples 
       N : number of items in a knapsack
       d : number of objectives  
       alpha: alpha values for gamma of length d
       beta: beta values for gamma of length d 
       R: d x d correlation matrix """
    L = np.linalg.cholesky(R)
    X_list = []
    for k in range(K):
        U = norm.cdf(np.random.randn(N, d) @ L.T)
        X = np.zeros(np.shape(U))
        for i in range(d):
            X[:, i] = gamma.ppf(U[:, i], a=alpha[i], scale=1.0 / beta[i])  # rate = 1/scale
        X_list.append(X)
    return X_list


In [3]:
def fit_gamma_marginals(X_list):
    X_stack = np.concatenate(X_list, axis=0)
    alpha = np.zeros(X_stack.shape[1])
    beta = np.zeros(X_stack.shape[1])
    for i in range(X_stack.shape[1]):
        a, loc, scale = gamma.fit(X_stack[:, i], floc=0.0)
        alpha[i] = a
        beta[i] = 1.0 / scale  # rate = 1/scale
    return alpha, beta



In [4]:
def transform_to_z(X, alpha, beta):
    n_samples, d = X.shape
    scales = 1.0 / beta
    U = np.zeros_like(X, dtype=float)
    for j in range(d):
        U[:, j] = gamma.cdf(X[:, j], a=alpha[j], scale=scales[j])
    U = np.clip(U, 1e-12, 1 - 1e-12)
    Z = np.zeros_like(U, dtype=float)
    for j in range(d):
        Z[:, j] = norm.ppf(U[:, j])
    return Z


In [5]:
def estimate_covariance(Z_list):
    """
    Estimate the (N*d x N*d) covariance and correlation.
    """
    Z_stacked = np.vstack([Z.ravel() for Z in Z_list])
    sigma = np.cov(Z_stacked, rowvar=False)
    R = np.corrcoef(Z_stacked, rowvar=False)
    
    return sigma, R

In [14]:
np.random.seed(42)
K, N, d = 200, 8, 2
alpha_true = np.array([3.0, 5.0])
beta_true  = np.array([2.0, 1.2])
R_true = np.array([[1.0, 0.4],
                   [0.4, 1.0]])

X_list = simulate_iid_gamma_gaussian(K, N, d, alpha_true, beta_true, R_true)
alpha_hat, beta_hat = fit_gamma_marginals(X_list)
Z_list = list()
for X in X_list:
    Z_trial = transform_to_z(X, alpha_hat, beta_hat)
    Z_list.append(Z_trial)
Sigma, R = estimate_covariance(Z_list)
R_hat = R[0:d,0:d]
print(R_hat)
print(alpha_hat)
print(beta_hat)
print(R[0:2*d,0:2*d])


[[1.         0.37322582]
 [0.37322582 1.        ]]
[3.15291868 5.01244767]
[2.11272872 1.18215583]
[[ 1.          0.37322582  0.01211136 -0.03246858]
 [ 0.37322582  1.          0.10524415  0.10483719]
 [ 0.01211136  0.10524415  1.          0.32982334]
 [-0.03246858  0.10483719  0.32982334  1.        ]]


In [12]:
np.random.seed(42)
K, N, d = 1000, 8, 3
alpha_true = np.array([3.0, 5.0, 4.0])
beta_true  = np.array([2.0, 1.2, 0.7])
R_true = np.array([[1.0, 0.4, -0.3],
                   [0.4, 1.0, 0.2], 
                   [-0.3, 0.2, 1]])

X_list = simulate_iid_gamma_gaussian(K, N, d, alpha_true, beta_true, R_true)
alpha_hat, beta_hat = fit_gamma_marginals(X_list)
Z_list = list()
for X in X_list:
    Z_trial = transform_to_z(X, alpha_hat, beta_hat)
    Z_list.append(Z_trial)
Sigma, R = estimate_covariance(Z_list)
R_hat = R[0:d,0:d]
print(R_hat)
print(alpha_hat)
print(beta_hat)
print(R[d:2*d,d:2*d])

[[ 1.          0.42028543 -0.31769038]
 [ 0.42028543  1.          0.19065613]
 [-0.31769038  0.19065613  1.        ]]
[2.97655614 4.95932924 3.99420914]
[1.99045212 1.18767201 0.69557792]
[[ 1.          0.38400755 -0.32309791]
 [ 0.38400755  1.          0.19932009]
 [-0.32309791  0.19932009  1.        ]]
